In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
import pandas as pd
from datetime import datetime

In [11]:
# Đọc dữ liệu từ các tệp CSV đã tải lên
transactions_df = pd.read_csv('/content/gdrive/MyDrive/FINALPROJECT/[Project1]_DL07_K302_NguyenManhDuc_NguyenBuiThanhTruc /Transactions.csv')
products_df = pd.read_csv('/content/gdrive/MyDrive/FINALPROJECT/[Project1]_DL07_K302_NguyenManhDuc_NguyenBuiThanhTruc /Products_with_Categories.csv')

In [44]:
print("Thông tin giao dịch:")
print(transactions_df.head())
print("\nThông tin sản phẩm:")
print(products_df.head())

# Loại bỏ cột cũ về sản phẩm nếu tồn tại

cols_to_remove = ['productName', 'Category', 'product_price']
existing_cols = [c for c in cols_to_remove if c in transactions_df.columns]
if existing_cols:
    print("Loại bỏ cột cũ gây xung đột:", existing_cols)
    transactions_df.drop(columns=existing_cols, inplace=True)

# Tính product price

products_df.rename(columns={'price': 'product_price'}, inplace=True)
products_subset = products_df[['productId','productName','product_price','Category']]

#  Merge thông tin sản phẩm
# Sử dụng suffixes để tránh xung đột nếu cột trùng
transactions_merged = transactions_df.merge(products_subset, on='productId', how='left', suffixes=('_trans','_prod'))

# Xử lý để tránh lặp cột
if 'Category_trans' in transactions_merged.columns and 'Category_prod' in transactions_merged.columns:
    # Tạo cột Category chung, ưu tiên dữ liệu 'prod' (hoặc combine_first tuỳ bạn)
    transactions_merged['Category'] = transactions_merged['Category_prod'].fillna(transactions_merged['Category_trans'])
    transactions_merged.drop(columns=['Category_trans','Category_prod'], inplace=True)
if 'productName_trans' in transactions_merged.columns and 'productName_prod' in transactions_merged.columns:
    transactions_merged['productName'] = transactions_merged['productName_prod'].fillna(transactions_merged['productName_trans'])
    transactions_merged.drop(columns=['productName_trans','productName_prod'], inplace=True)
if 'product_price_trans' in transactions_merged.columns and 'product_price_prod' in transactions_merged.columns:
    transactions_merged['product_price'] = transactions_merged['product_price_prod'].fillna(transactions_merged['product_price_trans'])
    transactions_merged.drop(columns=['product_price_trans','product_price_prod'], inplace=True)

# Tính total_spent

transactions_merged['total_spent'] = transactions_merged['items'] * transactions_merged['product_price']

# Hợp nhất các cột Category
category_cols = [c for c in transactions_df.columns if 'category' in c.lower()]
if 'Category' not in category_cols:
    # Nếu chưa có cột "Category" chuẩn, tạo nó
    transactions_df['Category'] = None
    # Cập nhật list cột
    category_cols = [c for c in transactions_df.columns if 'category' in c.lower() and c != 'Category']

for cat_col in category_cols:
    if cat_col != 'Category':
        transactions_df['Category'] = transactions_df['Category'].fillna(transactions_df[cat_col])

# Xoá cột thừa
category_cols_to_drop = [c for c in category_cols if c != 'Category']
transactions_df.drop(columns=category_cols_to_drop, inplace=True, errors='ignore')

print("\nSau merge, thông tin giao dịch:")
print(transactions_merged.head())

# Tính RFM

transactions_merged['Date'] = pd.to_datetime(transactions_merged['Date'], format='%d-%m-%Y')
most_recent_purchase = transactions_merged.groupby('Member_number')['Date'].max()
frequency = transactions_merged.groupby('Member_number').size()
monetary = transactions_merged.groupby('Member_number')['total_spent'].sum().round(2)

rfm_df = pd.DataFrame({
    'Recency': most_recent_purchase,
    'Frequency': frequency,
    'Monetary': monetary
})

today = pd.to_datetime('today')
rfm_df['Recency'] = (today - rfm_df['Recency']).dt.days

# Lưu các DataFrame vào thư mục trên Google Drive

output_path_rfm = '/content/gdrive/MyDrive/FINALPROJECT/[Project1]_DL07_K302_NguyenManhDuc_NguyenBuiThanhTruc /rfm.csv'
output_path_transactions = '/content/gdrive/MyDrive/FINALPROJECT/[Project1]_DL07_K302_NguyenManhDuc_NguyenBuiThanhTruc /Processed_transactions.csv'

transactions_df.to_csv(output_path_transactions, index=False)
rfm_df.to_csv(output_path_rfm, index=False)

print("\nDữ liệu đã được lưu vào Google Drive.")

Thông tin giao dịch:
   Member_number       Date  productId  items  total_spent  Category_x  \
0           1808 2015-07-21          1      3         23.4  Fresh Food   
1           2552 2015-01-05          2      1          1.8       Dairy   
2           2300 2015-09-19          3      3          9.0  Fresh Food   
3           1187 2015-12-12          4      3          2.4  Fresh Food   
4           3037 2015-02-01          2      1          1.8       Dairy   

   Category_y Category_trans  product_price_trans productName_trans  \
0  Fresh Food     Fresh Food                  7.8    tropical fruit   
1       Dairy          Dairy                  1.8        whole milk   
2  Fresh Food     Fresh Food                  3.0         pip fruit   
3  Fresh Food     Fresh Food                  0.8  other vegetables   
4       Dairy          Dairy                  1.8        whole milk   

  Category_prod  
0    Fresh Food  
1         Dairy  
2    Fresh Food  
3    Fresh Food  
4         Dairy  